# import 및 함수

In [1]:
import pandas as pd
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
wd = r'D:\3.자산\전산 dataset\230523'
event_dtype = {'채무자키':str, '계좌키':str, '법조치키':str, '관할법원코드':str, '타채무자키':str,'타법조치키':str}

In [2]:
# 만나이 계산기
def countAge(birth :str) -> int :
    if int(birth[:2]) <= (datetime.today().year - 2000) :
        year = int(birth[:2]) + 2000
    else : year = int(birth[:2]) + 1900
    month = int(birth[2:4])
    day = int(birth[4:])
    # 일단 나이를 계산합니다
    age = datetime.today().year - year
    if month > datetime.today().month:
        age -= 1
    elif month == datetime.today().month:
        if day > datetime.today().day:
            age -= 1
    return age

############ 채무상태 ###############
# 통합하기
def unite_debt_state (df_debt_ori, df_grt_ori) :
    """df를 받아서 담당자상태, 채무자상태, 보증인상태를 채무자를 기준으로 종합
    이후 is_out함수 호출"""
    채무자키 = df_debt_ori["채무자키"]
    채무자상태 = df_debt_ori["채무상태"]
    담당자상태 = df_debt_ori["담당자"]
    보증인수 = []
    보증인상태 = []
    제외여부 = []
    for key in tqdm(채무자키, total=len(채무자키), desc="채무상태 통합 중") :
        temp = df_grt_ori[df_grt_ori.채무자키==key] # 채무자 채무자키 하나씩 불러와서 보증인df
        보증인수.append(len(temp))
        보증인상태.append(temp.보증인상태.to_list())
    df_united = pd.DataFrame({"차주번호":채무자키, "채무자상태":채무자상태, "담당자상태":담당자상태, "보증인수":보증인수, "보증인상태":보증인상태})
    return df_united

# 대분류 넣기 - 주채무자의 행위가 개회, 파산은 보증인에 영향을 미치지 않음(법령확인함). 보증인의 행위는 주재무자에 영향 x
p_strongOut = re.compile(r"종결|매각|환매|개인회생\(면책\)|이민|회계파산") 
p_weakOut = re.compile(r"사망|시효|추심제외|포기채권")
p_신용회복 = re.compile(r"신용회복(?!\(실효\))")
p_개인회생 = re.compile(r"개인회생(?!\(폐지\))")
p_파산 = re.compile("파산") # 보증인 없는 경우에 적용하기 위함
p_실사 = re.compile(r"기타|말소|폐업|보증면탈|할당보류|서울|광주|일반유저")
p_보증인강종결 = re.compile(r"매각|종결|환매|이민|보증면탈|파산\(면책\)|파산면책|개인회생\(면책\)") 

def is_out_df(df_united) :
    """상태통합한 df를 주면 중분류 만들기"""

    채무상태구분 = []
    for i, v in tqdm(df_united.iterrows(), total=len(df_united.index), desc="out여부확인중") :
        if p_strongOut.search(v.채무자상태) or p_strongOut.search(v.담당자상태): # 종결류:언제나 제외
            채무상태구분.append("강제외")
        elif p_weakOut.search(v.채무자상태) or p_weakOut.search(v.담당자상태): # 종결류:내부적으로 다르게 취급하거나, 상황이 바뀌면 달라질 수 있음
            채무상태구분.append("약제외")
        elif p_신용회복.search(v.채무자상태) or p_신용회복.search(v.담당자상태):  
            채무상태구분.append("신복")
        elif p_개인회생.search(v.채무자상태) or p_개인회생.search(v.담당자상태):  
            채무상태구분.append("개회")
        elif p_파산.search(v.채무자상태) or p_파산.search(v.담당자상태):  
            if v.보증인수 == 0 :
                채무상태구분.append("파산제외")
            else :
                if all(map(lambda x : bool(p_보증인강종결.search(x)), v.보증인상태)) :
                    채무상태구분.append("파산제외")
                else :
                    채무상태구분.append("파산실사")
        elif p_실사.search(v.채무자상태) or p_실사.search(v.담당자상태): 
            채무상태구분.append("실사")
        else : 채무상태구분.append("정상")
    
    df_united["채무상태구분"] = 채무상태구분
    return df_united

In [3]:
######## 엑셀을 pickle파일로
# 여러건
# for root, dirs, files in os.walk(wd) :
#     for f in files :
#         if re.search(".xlsx", f) :
#             df = pd.read_excel(join(wd, f)).fillna("")
#             df.to_pickle(join(wd, os.path.splitext(f)[0]+".pkl"))

# 단일건
file = "개인회생새창_20230424_0921_기준데이터"
df = pd.read_excel(join(wd, file+".xlsx"), sheet_name="개인회생(진행)").fillna("")
df.to_pickle(join(wd, file+".pkl"))

In [4]:
# 파일 명칭 자동 저장
account_f, debt_f, grt_f, law_f, collection_f = "", "", "", "", ""
for root, dirs, files in os.walk(wd) :
    for f in files :
        if re.search(".pkl", f) :
            if re.search("계좌",f) : account_f = f
            elif re.search("채무자",f) : debt_f = f
            elif re.search("보증인",f) : grt_f = f
            elif re.search("법조치",f) : law_f = f
            elif re.search("입금",f) : collection_f = f

# 이거 하고 아래도 위 변수로 바꿔야 함 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [5]:
# 차주일반정보
df_debt_ori = pd.read_pickle(join(wd, "채무자.pkl")).fillna("")
df_grt_ori = pd.read_pickle(join(wd, "보증인.pkl")).fillna("")


In [6]:
df_debt = pd.DataFrame(None, columns=["차주번호", "차주구분", "주민번호", "연령", "최초원금", "최초가지급금", "최초미상환원금", "최초이자","최초채권액합계","회수합계", "원금잔액", "가지급금잔액", "미상환원금잔액", "이자잔액", "채권액합계", "지역", "연령구분", "금액구분", "지역구분", "매각사"])

In [7]:
# 보증인 
df_grt = pd.DataFrame(None, columns=["차주번호", "채권번호", "보증인번호", "보증인성별", "보증인연령"])

In [8]:
# 계좌
df_account_ori = pd.read_pickle(join(wd, "계좌.pkl")).fillna("")
df_account = pd.DataFrame(None, columns=["차주번호", "차주구분", "연령", "채권번호", "계정과목", "최초대출금액", "최초대출일", "대출만기일", "대출이율", "연체이율", "원금잔액", "가지급금잔액","미상환원금잔액", "이자잔액", "채권액합계", "시효완성일"])

In [5]:
# 법조치
df_event_ori = pd.read_pickle(join(wd, law_f), dtype=event_dtype).fillna("")
df_event = pd.DataFrame(None, columns=["채무자키", "계좌키", "신청일자", "법무구분", "관할법원", "사건번호", "진행상태", "청구금액법원", "청구금액"])

# 차주일반정보

### 기본 자료

In [9]:
df_debt.차주번호 = df_debt_ori.채무자키
df_debt.차주구분 = df_debt_ori.주민번호인.apply(lambda x : "법인" if re.search("-", x).start() == 3 else "개인")
df_debt.주민번호 = df_debt_ori.주민번호인
df_debt.연령 = df_debt_ori.연령 # 재수정해야함
df_debt.원금잔액 = df_debt_ori.현재원금
df_debt.가지급금잔액 = df_debt_ori.현재법비용
df_debt.미상환원금잔액 = df_debt[["원금잔액", "가지급금잔액"]].sum(axis=1)
df_debt.이자잔액 = df_debt_ori.loc[:,["현재미수이자", "현재연체이자"]].sum(axis=1)
df_debt.채권액합계 = df_debt[["미상환원금잔액", "이자잔액"]].sum(axis=1)
#df_debt.지역 = df_debt_ori.자택주소.apply(lambda x : x[:x.find(" ") + x[x.find(" ")+1:].find(" ")+1])
df_debt.최초원금 = df_debt_ori.최초원금
df_debt.최초가지급금 = df_debt_ori.최초법비용
df_debt.최초미상환원금 = df_debt_ori.OPB
df_debt.최초이자 = df_debt_ori.최초미수이자
df_debt.최초채권액합계 = df_debt_ori.최초합계
df_debt.회수합계 = df_debt_ori.회수합계
df_debt.매각사 = df_debt_ori.매각사

### 지역, 연령, 구분 정리

In [10]:
# 최초자료에서 지역 전처리해서 가져오기
지역, 연령 = [], []
for i, v in df_debt_ori[["자택주소", "자택번지인", "초본주소", "초본번지인", "직장주소", "직장번지인"]].iterrows() :
    자택주소 = re.sub("[^가-힣\s]", " ", v.자택주소.strip()).strip()
    자택번지인 = re.sub("[^가-힣\s]", " ", v.자택번지인.strip()).strip()
    초본주소 = re.sub("[^가-힣\s]", " ", v.초본주소.strip()).strip()
    초본번지인 = re.sub("[^가-힣\s]", " ", v.초본번지인.strip()).strip()
    직장주소 = re.sub("[^가-힣\s]", " ", v.직장주소.strip()).strip()
    직장번지인 = re.sub("[^가-힣\s]", " ", v.직장번지인.strip()).strip()

    if (자택주소 != "") or (자택번지인 != "") :
        주소 = 자택주소 + " " + 자택번지인
    elif (초본주소 != "") or (초본번지인 != "") :
        주소 = 초본주소 + " " + 초본번지인
    elif (직장주소 != "") or (직장번지인 != "") :
        주소 = 직장주소 + " " + 직장번지인
    else : 주소 = "불명"

    주소 = 주소.strip()

    if re.search(" ", 주소) :
        first = re.search(" ", 주소).start()
        if re.search(" ", 주소[first+1 : ]) :
            second = re.search(" ", 주소[first+1 : ]).start()
            지역.append(주소[:first+second+1])
        else :
            지역.append(주소)
    else :
        지역.append(주소)


df_debt.지역 = 지역

In [11]:
# 구분항목 추가하기
금액구분, 연령구분, 지역구분, 연령 = [], [], [], []
for i, v in df_debt.iterrows() :
    # 연령 재작업
    
    # 연령대구분
    if v.차주구분 == "법인" : 
        연령구분.append("법인")
        연령.append("법인")
    else : 
        연령.append(v.연령)
        연령대 = v.연령 // 10
        if 연령대 > 7 : 연령구분.append("70대 이상")
        elif 연령대 < 3 : 연령구분.append("20대")
        else : 연령구분.append(str(연령대)+"0대")

    # 금액구분
    if v.미상환원금잔액 <= 1000000 : 금액구분.append("1.1백")
    elif v.미상환원금잔액 <= 5000000 : 금액구분.append("2.5백")
    elif v.미상환원금잔액 <= 10000000 : 금액구분.append("3.1천")
    elif v.미상환원금잔액 <= 100000000 : 금액구분.append("4.1억")
    else : 금액구분.append("5.1억초과")

    # 지역구분
    지역 = v.지역[:2]
    if re.match(r"서울|강서|강남|노원", 지역) : 지역구분.append("1.서울")
    elif re.match(r"경기|인천|수원|부천|고양|하남|성남|화성|파주|의정|구리|용인", 지역) : 지역구분.append("2.인천,경기")
    elif re.match(r"충남|대전|충북|충청|세종|충주|청주|천안", 지역) : 지역구분.append("3.충청")
    elif re.match(r"전북|전라|광주|전남|군산|목포|전주", 지역) : 지역구분.append("4.전라")
    elif re.match(r"부산|경남|대구|경북|울산|경상|영천|창원", 지역) : 지역구분.append("5.경상")
    elif re.match(r"강원|강릉", 지역) : 지역구분.append("6.강원")
    elif re.match(r"제주", 지역) : 지역구분.append("7.제주")
    else : 지역구분.append("불명")

df_debt.연령 = 연령
df_debt.연령구분 = 연령구분
df_debt.금액구분 = 금액구분
df_debt.지역구분 = 지역구분

### 채무상태 정리

In [12]:
df_united = unite_debt_state(df_debt_ori, df_grt_ori)

채무상태 통합 중: 100%|██████████| 70796/70796 [00:09<00:00, 7222.17it/s]


In [13]:
df_united = is_out_df(df_united)

out여부확인중: 100%|██████████| 70796/70796 [00:03<00:00, 23175.21it/s]


In [14]:
df_debt = pd.merge(df_debt, df_united, on='차주번호', how="left")
len(df_debt)

70796

# 채권정보

In [15]:
df_account.차주번호 = df_account_ori.채무자키
df_account.차주구분 = df_account.차주번호.map(df_debt[["차주번호","차주구분"]].set_index('차주번호')['차주구분'])
df_account.연령 = df_account.차주번호.map(df_debt[["차주번호", "연령"]].set_index('차주번호')['연령'])
df_account.채권번호 = df_account_ori.계좌키
df_account.최초대출금액 = df_account_ori.은행대출원금
df_account.최초대출일 = df_account_ori.최초대출일
df_account.대출만기일 = df_account_ori.대출만기일
df_account.대출이율 = df_account_ori.이자율
df_account.연체이율 = df_account_ori.연체이자율
df_account.원금잔액 = df_account_ori.현재원금
df_account.가지급금잔액 = df_account_ori.현재법비용
df_account.미상환원금잔액 = df_account_ori[["현재원금", "현재법비용"]].sum(axis=1)
df_account.이자잔액 = df_account_ori[["현재미수이자", "현재연체이자"]].sum(axis=1)
df_account.채권액합계 = df_account[["미상환원금잔액", "이자잔액"]].sum(axis=1)
df_account.시효완성일 = df_account_ori.최초시효완성일

# 보증인정보

In [16]:
df_grt.차주번호 = df_grt_ori.채무자키
df_grt.채권번호 = df_grt_ori.계좌키
df_grt.보증인번호 = df_grt_ori.보증인키
df_grt.보증인성별 = df_grt_ori.주민번호인.str.replace("-", "").apply(lambda x : "법인" if len(x)==10 else "남자" if len(x)==13 and int(x[6])%2==1 else "여자" if len(x)==13 and int(x[6])%2==0 else "")
df_grt.보증인연령 = df_grt_ori.주민번호인.str.replace("-", "").apply(lambda x : "법인" if len(x)==10 else countAge(x[:6]) if len(x)==13 else "")

# 법조치정보
- df_event = pd.DataFrame(None, columns=["차주번호", "채권번호", "신청일자", "법무구분", "관할법원", "사건번호", "진행상태","청구금액법원", "청구금액"])

### 법조치 수정 후 (전산에 관할법원, 사건번호 수정한 뒤라면)

In [10]:
df_event = pd.DataFrame(None, columns=["채무자키", "계좌키", "신청일자", "법무구분", "관할법원", "사건번호", "사건구분", "진행상태","청구금액법원", "청구금액"])
df_event.채무자키 = df_event_ori.채무자키
df_event.계좌키 = df_event_ori.계좌키
df_event.신청일자 = df_event_ori.접수일
df_event.법무구분 = df_event_ori.법조치구분
df_event.관할법원 = df_event_ori.관할법원
df_event.사건번호 = df_event_ori.사건번호.str.replace(" ", "")
df_event.사건구분 = df_event_ori.사건구분
df_event.진행상태 = df_event_ori.종국결과
df_event.청구금액법원 = df_event_ori.청구금액법원
df_event.청구금액 = df_event_ori.청구금액
print(len(df_event.index))

154959


In [8]:
####################
# 중복제거전 저장
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치_중복미제거.pkl"))
#df_event.to_excel(join(wd, "전처리완료", "전처리_법조치_중복미제거.xlsx"), index=False)
####################

In [11]:
# 사건번호 없는 거 제거
df_event = df_event.query('사건번호 not in ["",0,"0"]')
print(len(df_event.index))

# 중복제거
# keep="first" 기본값 입력된 순서이기 때문에 제일 먼저 입력한 값이 남는다.
# 사건번호만 불러올 때는 이것이 의미 있으나, 청구금액 같은 것을 볼 때는 중복제거 안 된 것을 참고해야함.
df_event.drop_duplicates(subset=["채무자키", "관할법원","사건번호"], inplace=True, ignore_index=True) 
print(len(df_event.index))

# 확실히 법원 아닌 거 제거
df_event = df_event.query('~관할법원.str.contains("경찰|검찰|법무|법률|세무|차량|구청(?!도)|공사|자산|지급|카드|보증")')
print(len(df_event.index))

# # 확실히 제외할 법무구분
# df_event = df_event.query('~관할법원.str.contains("집행문부여")')
# print(len(df_event.index))

149343
102454
102330


In [12]:
####################
# 중복제거 후 저장
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치_중복제거.pkl"))
df_event.to_excel(join(wd, "전처리완료", "전처리_법조치_중복제거.xlsx"), index=False)
####################

### 법조치 수정 전

In [5]:
df_event.채무자키 = df_event_ori.채무자키
df_event.계좌키 = df_event_ori.계좌키
df_event.신청일자 = df_event_ori.접수일
df_event.법무구분 = df_event_ori.법조치구분
df_event.관할법원 = df_event_ori.관할법원
df_event.사건번호 = df_event_ori.사건번호
df_event.진행상태 = df_event_ori.종국결과

In [6]:
df_event.청구금액 = df_event_ori.청구금액
# 청구금액 = []
# for i, v in df_event_ori[["청구금액법원", "청구금액"]].iterrows() :
#     if v.청구금액법원 > 0 : 청구금액.append(v.청구금액법원)
#     else : 청구금액.append(v.청구금액)
# df_event.청구금액 = 청구금액

In [7]:
pwd = r"C:\Users\SL\Desktop\법조치 수정"
df_reg = pd.read_excel(join(pwd, "법원 목록 정리.xlsx")).fillna("")
print(len(df_reg))
# dict만들기
cnt = 0
reg1, reg2, reg3, regF1, regF2 = {}, {}, {}, {}, {}
for i, v in df_reg.iterrows() :
    if v.비고 == "가정" : # 지법에는 아무 값도 없음
        if v.시군 != "" :
            regF1[v.시군] = v.법원
        if v.지원 != "" :
            regF2[v.지원] = v.법원        
    else :
        if v.시군 != "" :  
            reg1[v.시군] = v.법원
        if v.지원 != "" :
            reg2[v.지원] = v.법원
        if v.지법 != "" :
            reg3[v.지법] = v.법원
print(len(reg1), len(reg2), len(reg3), len(regF1), len(regF2))

204
122 45 14 22 8


In [55]:
# 관할법원
for i, v in tqdm(df_event.iterrows(), total=len(df_event)) :
    out_for_continue = False    ########################### no Touch

    # 검찰, 경찰 등 제외하기 # 사건번호, 대상자명도 수정해야 하니 전체목록을 대상으로 해야한다. 따라서 
    if re.search("경찰|검찰|법무|법률|세무|차량|구청(?!도)|공사|자산|미입력|지급|카드|보증", v.관할법원) or (v.관할법원 == ""):  # 대'구청'도군법원유의
        df_event.loc[i, "관할법원"] = v.관할법원
        continue
        
    # 가정 법원인지 확인
    if re.search("가정", v.관할법원) :
        for exp, right_court_name in regF1.items() :
            if re.search(exp, v.관할법원) :
                df_event.loc[i, "관할법원"] = right_court_name
                out_for_continue = True
                break
        if out_for_continue : continue # regF1에서 수정할 법원 나왔으면 새로운 반복문 시작
        # regF2 확인하기
        for exp, right_court_name in regF2.items() :
            if re.search(exp, v.관할법원) :
                df_event.loc[i, "관할법원"] = right_court_name
                out_for_continue = True
                break

        if out_for_continue == False : # '가정법원'에 들어왔음에도, 조건에 걸린 게 없어서 수정한 바가 없음
            df_event.loc[i, "관할법원"] = v.관할법원
        continue # 가정법원에 걸렸으니 무조건 여기서 끝냄

    else :
        # 가정법원 아닌 경우 : reg1먼저 확인하기 
        for exp, right_court_name in reg1.items() : # 
            if re.search(",", exp) :
                compile_items = exp.split(",")
                if re.search(compile_items[0], v.관할법원) and re.search(compile_items[1], v.관할법원) : #항상 2개
                    df_event.loc[i, "관할법원"] = right_court_name
                    out_for_continue = True
                    break
            else :
                if re.search(exp, v.관할법원) :
                    df_event.loc[i, "관할법원"] = right_court_name
                    out_for_continue = True
                    break
        if out_for_continue : continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

        # reg2 확인하기
        for exp, right_court_name in reg2.items() :
            if re.search(exp, v.관할법원) :
                df_event.loc[i, "관할법원"] = right_court_name
                out_for_continue = True
                break
        if out_for_continue : continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

        # reg3 확인하기
        for exp, right_court_name in reg3.items() :
            if re.search(exp, v.관할법원) :
                df_event.loc[i, "관할법원"] = right_court_name
                out_for_continue = True
                break

        if out_for_continue ==False: # 여기까지 왔는데도 바뀐 게 없다면 ..
            df_event.loc[i, "관할법원"] = v.관할법원
        continue # reg1에서 수정할 법원 나왔으면 새로운 반복문 시작

100%|██████████| 153642/153642 [01:56<00:00, 1319.05it/s]


In [57]:
# 사건번호
p_event=re.compile(r"(\d{2,4})[\s,]?([가-힣]{1,4})[\s-]?(\d+)") # year = [1], sign = [2], sn=[3]
for i, v in tqdm(df_event.iterrows(), total=len(df_event)) :
    res = p_event.search(v.사건번호)

    if res != None :
        year = res[1]
        event = year + res[2] + res[3]
        # 연도 4자리고, 전후가 동일하면 pass, 수정여부에 x
        if len(year) == 4 : pass
        # 연도 2자리고 00 ~ 23사이면 20 붙이고, 아니면 19붙이기, 수정여부에 ㅇ
        elif len(year) == 3 :
            if int(year) >= 200 and int(year) <=223 :
                df_event.loc[i,"사건번호"] = event[0] + "0" + event[1:]
            else : pass
        elif len(year) == 2 :
            if int(year) <= 23 :
                df_event.loc[i,"사건번호"] = "20" + event
            elif int(year) > 70  :
                df_event.loc[i,"사건번호"] = "19" + event
        else : pass

100%|██████████| 153642/153642 [00:05<00:00, 26194.29it/s]


In [59]:
# 중복제거
df_event.drop_duplicates(subset=["차주번호", "관할법원","사건번호"], inplace=True, ignore_index=True)

# 저장

In [54]:
# 엑셀저장
# df_debt.to_excel(join(wd, "전처리완료", "전처리_채무자.xlsx"), index=False)
# df_grt.to_excel(join(wd, "전처리완료", "전처리_보증인.xlsx"), index=False)
# df_account.to_excel(join(wd, "전처리완료", "전처리_계좌.xlsx"), index=False)
df_event.to_excel(join(wd, "전처리완료", "전처리_법조치.xlsx"), index=False)

In [57]:
# pkl 저장
# df_debt.to_pickle(join(wd, "전처리완료", "전처리_채무자.pkl"))
# df_grt.to_pickle(join(wd, "전처리완료", "전처리_보증인.pkl"))
# df_account.to_pickle(join(wd, "전처리완료", "전처리_계좌.pkl"))
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치.pkl"))

# 채무자키를 기준으로 정리

In [61]:
keys = pd.read_excel(r"C:\Users\SL\Desktop\솔림헬프 추가자료요청.xlsx", sheet_name="Pool8무담보리스트")["채무자키"]

In [62]:
result = pd.merge(keys, df_event)
result

,채무자키,계좌키,신청일자,법무구분,관할법원,사건번호,사건구분,진행상태,청구금액법원,청구금액
0,20407793,200920874,2015-08-13,지급명령,서울서부지방법원,2015가소493293,가소,원고승,0,2549135
1,20407796,200924023,2017-10-10,지급명령,서울남부지방법원,2017차전165769,차전,,0,11458461
2,20407796,200924023,2017-12-08,압류및추심,광주지방법원,2017타채40229,타채,,0,11678948
3,20407797,200925502,2019-09-09,지급명령,광주지방법원,2019차전16068,차전,지급명령,9249817,9251991
4,20407797,200925502,2021-09-09,압류및추심,부산지방법원,2021타채69992,타채,인용,0,10101892
...,...,...,...,...,...,...,...,...,...,...
44858,20458716,200940996,2020-03-18,지급명령,부산지방법원,2018차전13101,차전,소제기,0,0
44859,20458716,200940996,2020-03-18,임의경매,부산지방법원,2018타경5491,타경,,0,0
44860,20458748,200941157,2020-03-18,개인회생,의정부지방법원,2019개회6732,개회,,0,0
44861,20458756,200941208,,파산,창원지방법원,2020하단465,하단,,0,6759422


In [63]:
result.to_excel(join(wd, "POOL8_법조치.xlsx"), index=False)

# 특정 매각사만 정리
- 1: 정상 // 2:  파산실사, 실사 // 3. 신복, 개회, 파산제외 // 4 : 강제외, 약제외  

In [ ]:
# 1. 채무자에서 특정매각사, 채무상태를 필터링 하여 채무자키 선별
# 2. isin()이나 set_index()를 사용하여 계좌 ~ 법조치 정보 불러오기
# 3. 하나의 파일 각각의 셀에 저장하기

In [26]:
# 파일 읽기
df_debt = pd.read_pickle(join(wd, "전처리완료", "전처리_채무자.pkl"))
df_grt = pd.read_pickle(join(wd, "전처리완료", "전처리_보증인.pkl"))
df_account = pd.read_pickle(join(wd, "전처리완료", "전처리_계좌.pkl"))
# df_event = pd.read_pickle(join(wd, "전처리완료", "전처리_법조치.pkl"))

In [3]:
df_debt.columns

Index(['차주번호', '차주구분', '연령', '최초원금', '최초가지급금', '최초미상환원금', '최초이자', '최초채권액합계',
       '회수합계', '원금잔액', '가지급금잔액', '미상환원금잔액', '이자잔액', '채권액합계', '지역', '연령구분',
       '금액구분', '지역구분', '매각사', '채무자상태', '담당자상태', '보증인수', '보증인상태', '채무상태구분'],
      dtype='object')

In [28]:
########################################
매각사 = "전체"
########################################
filename = 매각사 + " 써머리.xlsx"

# 1. 매각사 조건
result = df_debt#df_debt[df_debt.매각사.str.contains(매각사)]
len(result)

70796

In [21]:
result[["주민번호"]].value_counts()

주민번호          
861014-1386915    8
780505-1852415    7
680207-2634714    7
740520-1273313    6
860214-2824310    6
                 ..
700605-1101021    1
700610-1095216    1
700622-1279214    1
700708-2648114    1
970228-2164314    1
Length: 1003, dtype: int64

In [22]:
result.to_excel(r"D:\0.전산\2.이관\써머리\data_230127\전체.xlsx", index=False)

In [29]:
# 2. 채무상태구분 기본
# 정상
result_정상 = result.query('채무상태구분=="정상"')
# 실사
result_실사 = result.query('채무상태구분=="파산실사" or 채무상태구분=="실사"')
# 신회파
result_신회파 = result.query('채무상태구분=="신복" or 채무상태구분=="개회" or 채무상태구분=="파산제외"')
# 강제외, 약제외
result_제외 = result.query('채무상태구분=="강제외" or 채무상태구분=="약제외"')
print(len(result_정상), len(result_실사), len(result_신회파), len(result_제외))

30022 2786 24408 13580


In [24]:
path = join(r"D:\0.전산\2.이관\써머리", 매각사)
if not os.path.exists(path) : os.mkdir(path)
if not os.path.exists(join(path, filename)) : pd.DataFrame().to_excel(join(path, filename))

In [25]:
with pd.ExcelWriter(join(path, filename), engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    result_정상.to_excel(writer, sheet_name = "정상", index=False)
    result_실사.to_excel(writer, sheet_name = "실시필요", index=False)
    result_신회파.to_excel(writer, sheet_name = "신복회생파산", index=False)
    result_제외.to_excel(writer, sheet_name = "제외", index=False)

In [26]:
result_정상.주민번호.value_counts()

861014-1386915    8
780505-1852415    7
680207-2634714    7
740520-1273313    6
860214-2824310    6
                 ..
721025-1448813    1
780828-2536112    1
700128-1350411    1
710308-1581117    1
700921-1830519    1
Name: 주민번호, Length: 815, dtype: int64

In [ ]:
# 병합위한 채무자키 시리즈
채무자키 = result_debt.채무자키
print(len(채무자키))
# 보증인, 계좌, 사건번호 시트 만들기
result_grt = pd.merge(채무자키, df_grt, how='left')
result_account = pd.merge(채무자키, df_account, how='left')
result_event = pd.merge(채무자키, df_event, how='left')
print(len(result_grt, result_account, result_event))

In [ ]:
#######################################
path = r"D:\0.전산\2.이관\써머리\BNK"
#######################################
if not os.path.exists(path) : os.mkdir(path)
if not os.path.exists(join(path, filename)) : pd.DataFrame().to_excel(join(path, filename))

In [ ]:

with pd.ExcelWriter(join(path, filename), engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    result_debt.to_excel(writer, sheet_name = "A.차주일반정보", index=False)
    result_account.to_excel(writer, sheet_name = "B.채권정보", index=False)
    result_grt.to_excel(writer, sheet_name = "C.보증인정보", index=False)
    result_event.to_excel(writer, sheet_name = "D.법조치정보", index=False)